# Razonamiento de Cadena de Pensamiento

## Configuración

In [1]:
import os
import openai
import sys
from openai import OpenAI

from helper import get_openai_api_key
openai.api_key = get_openai_api_key()
client = OpenAI(api_key=openai.api_key) 


In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, max_tokens=500):
    response = client.chat.completions.create( 
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content

## Prompting de Cadena de Pensamiento

In [3]:
delimiter = "####"
system_message = f"""
Sigue estos pasos para responder a las consultas de los clientes.
La consulta del cliente estará delimitada con cuatro hashtags,\
es decir, {delimiter}.

Paso 1:{delimiter} Primero decide si el usuario está \
preguntando sobre un producto o productos específicos. \
La categoría de producto no cuenta.

Paso 2:{delimiter} Si el usuario pregunta sobre \
productos específicos, identifica si \
los productos están en la siguiente lista.
Todos los productos disponibles:
1. Producto: TechPro Ultrabook
   Categoría: Computadoras y Laptops
   Marca: TechPro
   Número de Modelo: TP-UB100
   Garantía: 1 año
   Calificación: 4.5
   Características: Pantalla de 13.3 pulgadas, 8GB RAM, 256GB SSD, procesador Intel Core i5
   Descripción: Un ultrabook elegante y ligero para el uso diario.
   Precio: $799.99

2. Producto: BlueWave Gaming Laptop
   Categoría: Computadoras y Laptops
   Marca: BlueWave
   Número de Modelo: BW-GL200
   Garantía: 2 años
   Calificación: 4.7
   Características: Pantalla de 15.6 pulgadas, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Descripción: Una laptop para juegos de alto rendimiento para una experiencia inmersiva.
   Precio: $1199.99

3. Producto: PowerLite Convertible
   Categoría: Computadoras y Laptops
   Marca: PowerLite
   Número de Modelo: PL-CV300
   Garantía: 1 año
   Calificación: 4.3
   Características: Pantalla táctil de 14 pulgadas, 8GB RAM, 256GB SSD, bisagra de 360 grados
   Descripción: Una laptop convertible versátil con una pantalla táctil sensible.
   Precio: $699.99

4. Producto: TechPro Desktop
   Categoría: Computadoras y Laptops
   Marca: TechPro
   Número de Modelo: TP-DT500
   Garantía: 1 año
   Calificación: 4.4
   Características: Procesador Intel Core i7, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Descripción: Una potente computadora de escritorio para trabajar y jugar.
   Precio: $999.99

5. Producto: BlueWave Chromebook
   Categoría: Computadoras y Laptops
   Marca: BlueWave
   Número de Modelo: BW-CB100
   Garantía: 1 año
   Calificación: 4.1
   Características: Pantalla de 11.6 pulgadas, 4GB RAM, 32GB eMMC, Chrome OS
   Descripción: Un Chromebook compacto y asequible para las tareas diarias.
   Precio: $249.99

Paso 3:{delimiter} Si el mensaje contiene productos \
de la lista anterior, enumera cualquier suposición que el \
usuario esté haciendo en su \
mensaje, por ejemplo, que la Laptop X es más grande que \
la Laptop Y, o que la Laptop Z tiene una garantía de 2 años.

Paso 4:{delimiter}: Si el usuario hizo alguna suposición, \
averigua si la suposición es cierta basándote en la \
información de tu producto.

Paso 5:{delimiter}: Primero, corrige amablemente las \
suposiciones incorrectas del cliente si corresponde. \
Solo menciona o haz referencia a los productos de la lista de \
5 productos disponibles, ya que estos son los únicos 5 \
productos que vende la tienda. \
Responde al cliente en un tono amigable.

Usa el siguiente formato:
Paso 1:{delimiter} <razonamiento del paso 1>
Paso 2:{delimiter} <razonamiento del paso 2>
Paso 3:{delimiter} <razonamiento del paso 3>
Paso 4:{delimiter} <razonamiento del paso 4>
Respuesta al usuario:{delimiter} <respuesta al cliente>

Asegúrate de incluir {delimiter} para separar cada paso.
"""

In [4]:
user_message = f"""
por cuánto es más cara la Chromebook BlueWave \
que la computadora de escritorio TechPro"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

#### Paso 1: El usuario está preguntando sobre dos productos específicos: la Chromebook BlueWave y la computadora de escritorio TechPro.

#### Paso 2: Ambos productos están en la lista proporcionada.

#### Paso 3: El usuario está asumiendo que la Chromebook BlueWave es más cara que la computadora de escritorio TechPro.

#### Paso 4: La Chromebook BlueWave tiene un precio de $249.99, mientras que la computadora de escritorio TechPro tiene un precio de $999.99. Por lo tanto, la Chromebook BlueWave es $750 más barata que la computadora de escritorio TechPro.

#### Respuesta al usuario: La Chromebook BlueWave es $750 más barata que la computadora de escritorio TechPro.


In [5]:
user_message = f"""
venden televisores"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

#### Paso 1: El usuario está preguntando sobre un tipo de producto que no está en la lista de productos disponibles.

Respuesta al usuario: Lo siento, en nuestra tienda actualmente no vendemos televisores. ¿Puedo ayudarte con información sobre alguno de nuestros productos disponibles?


## Pensamiento interno
- Como le pedimos al LLM que separe sus pasos de razonamiento con un delimitador, podemos ocultar el razonamiento de la cadena de pensamiento del resultado final que ve el usuario.

In [6]:
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Lo siento, estoy teniendo problemas en este momento, por favor intenta hacer otra pregunta."
    
print(final_response)

Paso 1: El usuario está preguntando sobre un tipo de producto que no está en la lista de productos disponibles.

Respuesta al usuario: Lo siento, en nuestra tienda actualmente no vendemos televisores. ¿Puedo ayudarte con información sobre alguno de nuestros productos disponibles?
